In [63]:
import json
import pandas as pd
import json
import requests

BASE_URL = "https://yfapi.net"
API_KEY = "KDesH3t06S7XJo4r5f6qO24NhrK3uYMT1V9ErK8j"

headers = {
    'x-api-key': API_KEY
}

# Function to get stock details
def get_stock_quote(ticker):
    url = f"{BASE_URL}/v6/finance/quote"
    params = {
        'symbols': ticker
    }
    try:
        response = requests.request("GET", url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        stock_info = {
            "Ticker": data['quoteResponse']['result'][0]['symbol'],
            "Full Name": data['quoteResponse']['result'][0]['longName'],
            "Current Market Price": data['quoteResponse']['result'][0]['regularMarketPrice'],
            "52 Week High": data['quoteResponse']['result'][0]['fiftyTwoWeekHigh'],
            "52 Week Low": data['quoteResponse']['result'][0]['fiftyTwoWeekLow']
        }
        return stock_info
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

# Function to get target mean price
def get_target_mean_price(ticker):
    url = f"{BASE_URL}/v11/finance/quoteSummary/{ticker}"
    params = {
        'modules': 'financialData'
    }
    try:
        response = requests.request("GET", url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
        target_mean_price = data['quoteSummary']['result'][0]['financialData']['targetMeanPrice']['raw']
        return target_mean_price
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

# Function to get trending stocks
def get_trending_stocks():
    url = f"{BASE_URL}/v1/finance/trending/US"
    try:
        response = requests.request("GET", url, headers=headers)
        response.raise_for_status()
        data = response.json()
        trending_stocks = data['finance']['result'][0]['quotes'][:5]
        return trending_stocks
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

# Main function
def main():
    ticker = input("Enter the stock ticker symbol: ").upper()

    stock_info = get_stock_quote(ticker)

    if stock_info:
        stock_info['Target Mean Price'] = get_target_mean_price(ticker)
        
        print("\nStock Information:")
        for key, value in stock_info.items():
            print(f"{key}: {value}")

        trending_stocks = get_trending_stocks()
        print("\nTop 5 Trending Stocks:")
        for stock in trending_stocks:
            print(stock['symbol'])

        df = pd.DataFrame([stock_info])
        df.to_csv("stock_info.csv", index=False)
        print("\nStock data saved to stock_info.csv")

    else:
        print("Unable to get stock data. Check the ticker symbol or try again.")

main()

Enter the stock ticker symbol:  AAPL



Stock Information:
Ticker: AAPL
Full Name: Apple Inc.
Current Market Price: 227.725
52 Week High: 237.23
52 Week Low: 164.08
Target Mean Price: 240.58

Top 5 Trending Stocks:
^N225
NQ=F
ES=F
AMSC
BTC-USD

Stock data saved to stock_info.csv
